# Homework 1 : CUAHSI VU : Snow Hydrology

This notebook will be relatively short because this where the real work starts and most of the notebook will be written by you (or at least that is the plan). We'll do three exercises in homework 1:

* Homework 1 exercise 1: Change the snow layering scheme

* Homework 1 exercise 2: Change the albedo formulation and albedo parameters

* Homework 1 exercise 3: Change the parameters controlling drainage through the snow pack

We'll use the setup from the `homework_1_C_setup.ipynb` notebook for all three exercises.

As before, remember that you want to save your notebooks in a place other than the `examples` directory tree, because files you save there may be deleted and overwritten the next time you start your virtual machine. Save your files somewhere in the `work` directory tree: `work/homework_1` would be a great place for them. So click on "_File --> Save Notebook As..._" and save this notebook as `work/homework_1/homework_1_D_exercise_1.ipynb` before you continue. If things go hopelessly wrong, you can always start over again with the original notebook in your `examples` directory tree.

## D. Homework 1 Exercise 1

### Setup

We'll import some of the python modules that you are likely to need right up front. Note that you only need to do this once in a notebook. As long as you are working in the same session, the modules will remain loaded for the duration of that session.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt # if you want to do some plotting using matplotlib (optional)
from pysumma.Simulation import Simulation # create Simulation objects
from pysumma.hydroshare import hydroshare # create hydroshare objects to upload to HydroShare
from pysumma.LocalParamInfo import LocalParamInfo # modify local parameters. Explained below

We'll also set some variables to make sure that everyone is using the same setup. This uses the setup that you installed in the `homework_1_C_setup.ipynb` notebook.

In [ ]:
resource_id = '363296896e9d48d9b464d766fcd3073a'
case_path = '/home/jovyan/work/{}/{}/data/contents/reynolds'.format(resource_id, resource_id)
filemanager_path = case_path+'/settings/snow_fileManager_template.txt'
summa_code_path = '/home/jovyan/summa'

### Change the snow layering scheme

Snow models represent a snow pack as one or more layers of snow and keep track of how much water is stored in each layer (snow water equivalent or SWE) as well as other state variables such as temperature. The model then tracks the time evolution of these state variables in each layer as well as the fluxes of water and energy between the layers.

In SUMMA you can choose between two different methods to combine and sub-divide snow layers. This behavior is controlled by the `snowLayers` variable in the [SUMMA decisions file](https://summa.readthedocs.io/en/latest/input_output/SUMMA_input/#infile_model_decisions) and can take one of two values, either `CLM_2010` or `jrdn1991`. The first refers to how snow layers were handled in the Community Land Model (CLM), while the second refers the snow layering scheme in SNTHERM as describe in  Jordan (1991).

Among others things, the `CLM_2010` only has a limited number of snow layers. The layers store a maximum amount of SWE and when SWE in a layer exceeds that value, a new layer is created. Similarly, when SWE in a layer falls below a minimum SWE, the layer is merged with a neighboring layer. Once the maximum number of layers has been created, the deepest layer can grow as deep as it needs to be and acts as snow reservoir with unlimited capacity. The `jrdn1991` scheme on the other hand keeps creating additional layers as the snow pack grows in depth.

Here we will do four different simulations:

* SUMMA with `CLM_2010`
* SUMMA with `jrdn1991`
* SUMMA with `CLM_2010`, but we will configure it to act as a two layer model (the default version uses five snow layers)
* SUMMA with `CLM_2010`, a two layer model and a thick top layer.

So now it is up to you. Just to refresh your memory we'll show you how to get started and then will provide some hints along the way.

#### SUMMA with CLM_2010

Create a simulation object. The path for the SUMMA code and the file manager (the SUMMA input file with information about all the other input files) are already defined above. We can use the same file manager file (and thus the same simulation object) for all the runs, just keep track of what you did. We also already imported the `Simulation()` method.

In [ ]:
S = Simulation(case_data=filemanager_path, summa_code=summa_code_path)

Set the decision for `snowLayers` to `CLM_2010`.

In [ ]:
S.decision_obj.snowLayers.options

In [ ]:
S.decision_obj.snowLayers.value = 'CLM_2010'

Use a `run_suffix` that uniquely identifies this run so that you can keep your runs apart, for example `CLM_2010`, and run the model. If you assign the results to its own dataset (e.g. `ds_clm_2010`) and output file (e.g. `file_clm_2010`) it will be easier to gather all the files and upload them to HydroShare (or you can do so one by one, this is all up to you). The run itself will take about a minute.

In [ ]:
ds_clm_2010, file_clm_2010 = S.execute(run_suffix='CLM_2010', run_option='local')

You can quickly check how the number of snow layers changes by plotting

In [ ]:
ds_clm_2010.nSnow.isel(hru=0).plot()

As you can see, the number of layers never exceeds five.

#### SUMMA with jrdn1991

Repeat the exercise with the `jrdn1991` option for `snowLayers`. Examine the differences in the snow layers themselves, but also in other characteristics of snow pack evolution. In SUMMA, all other options will be the same between these two simulations unless you change things explicitly.

#### SUMMA with CLM_2010 configured as a  two layer model

Now change the `snowLayers` option back to `CLM_2010` and configure the model to act as a two layer snow model rather than a five layer snow model. Two layer models are quite common. The surface layer is typically quite thin in these setups to respond quickly to changing atmospheric conditions, while the second layer acts as a snow reservoir with unlimited capacity.

To configure the snow model as a two layer model, we need to change some model parameters, something we have not done so far, so we need to introduce another function in pySUMMA. Here, we'll change some settings in the SUMMA input file that contains parameter information (the name of that file is stored in your simulation object as `S.local_par.filename`).


In [ ]:
S.local_par.filename

We are going to change the file so it has the following settings:
```
zminLayer3                |     100.0000 |       0.0500 |       0.0500
zminLayer4                |     100.0000 |       0.1000 |       0.1000
zminLayer5                |     100.0000 |       0.2500 |       0.2500
zmaxLayer2_lower          |    1000.0000 |       0.2000 |       0.2000
zmaxLayer3_lower          |    1000.0000 |       0.5000 |       0.5000
zmaxLayer4_lower          |    1000.0000 |       1.0000 |       1.0000
zmaxLayer2_upper          |    1000.0000 |       0.1500 |       0.1500
zmaxLayer3_upper          |    1000.0000 |       0.3000 |       0.3000
zmaxLayer4_upper          |    1000.0000 |       0.7500 |       0.7500
```
For now don't worry about the last two columns (they are not used). The value in the second column determines when a new snow layer is created or when two layers are merged. By creating a second layer with a very large capacity, layers 3, 4, and 5 will never be created and the model acts like a two layer snow model.

While you can make these changes directly in the text file (`S.local_par.filename`), we will use pySUMMA to do so. This works somewhat differently than for the decisions (pySUMMA is a work in progress), but you can do the following:

In [ ]:
S.decision_obj.snowLayers.value = 'CLM_2010'

local_params_file = S.file_dir + '/' + S.local_par.filename
local_params = LocalParamInfo(local_params_file)

local_params.replace('zminLayer3', 100.0000, 0.0500, 0.0500)
local_params.replace('zminLayer4', 100.0000, 0.1000, 0.1000)
local_params.replace('zminLayer5', 100.0000, 0.2500, 0.2500)
local_params.replace('zmaxLayer2_lower', 1000.0000, 0.2, 0.2000)
local_params.replace('zmaxLayer3_lower', 1000.0000, 0.5000, 0.5000)
local_params.replace('zmaxLayer4_lower', 1000.0000, 1.0000, 1.0000)
local_params.replace('zmaxLayer2_upper', 1000.0000, 0.1500, 0.1500)
local_params.replace('zmaxLayer3_upper', 1000.0000, 0.3000, 0.3000)
local_params.replace('zmaxLayer4_upper', 1000.0000, 0.7500, 0.7500)

Now go ahead and run the model. After you run the model, it is a good idea to restore the original parameters, which you can do by executing `local_params.restore()`. Note that this will only restore the immediately previous set of parameters. If things go really wrong, you can rerun the setup notebook (`work/homework_1/homework_1_C_setup.ipynb`) and start with a clean slate.

Before you go on, verify that your model run only uses two snow layers and restore the original parameters by executing `local_params.restore()`.

#### SUMMA with CLM_2010 configured as a two layer model with a thick top layer

One more run with `CLM_2010`, two layers, and a thick top layer. To do this, change the local parameters to
```
zminLayer1                |       0.0500 |       0.0500 |       0.0500
zminLayer2                |       0.0200 |       0.0500 |       0.0500
zminLayer3                |     100.0000 |       0.0500 |       0.0500
zminLayer4                |     100.0000 |       0.1000 |       0.1000
zminLayer5                |     100.0000 |       0.2500 |       0.2500
zmaxLayer1_lower          |       0.2000 |       0.0500 |       0.0500
zmaxLayer2_lower          |    1000.0000 |       0.2000 |       0.2000
zmaxLayer3_lower          |    1000.0000 |       0.5000 |       0.5000
zmaxLayer4_lower          |    1000.0000 |       1.0000 |       1.0000
zmaxLayer1_upper          |       0.1500 |       0.0300 |       0.0300
zmaxLayer2_upper          |    1000.0000 |       0.1500 |       0.1500
zmaxLayer3_upper          |    1000.0000 |       0.3000 |       0.3000
zmaxLayer4_upper          |    1000.0000 |       0.7500 |       0.7500

```

and run again. After you are done, restore the local parameters to the original values, because we will use the same setup in the next notebook.

## On to the next step

That's it for the first part of the first homework as far as the model simulations go. Prof. Lundquist will talk more about the analysis that you need to do on these simulations. You can do the analysis here in this notebook if python is your analysis software of choice or you can upload your output to HydroShare (in NetCDF or csv format) to do additional analysis.

Save this notebook and close the tab. You can also right-click on the file in the left panel if it has a green dot next to it and select "_Shutdown kernel_" from the popup menu to stop the python session that is executing the commands in this notebook. Then open the next notebook: 
`work/homework_1/homework_1_E_exercise_2.ipynb`.

## References

* Jordan, R., 1991: A one-dimensional temperature model for a snow cover. Technical documentation for SNTHERM.89. U.S. Army Corps of Engineers Cold Regions Research and Engineering Laboratory, 49 pp.